# ***Install dependencies***

In [15]:

!pip install -qU langchain langchain-google-genai langchain-community requests
!pip install -qU langchain langchain-google-genai requests





**What this does**

Installs the Python packages you need: langchain, the Google GenAI connector, and requests.

The second line is redundant — you only need to run one install line. It won’t hurt, but it’s unnecessary.

**Why it matters**

LangChain provides the high-level agent/tool API.

langchain-google-genai lets you call Gemini via Google’s API.

requests is used to call the News API.

# ***Load secret keys from Colab userdata***

In [19]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('RAGAGENTKEY')
os.environ['NEWS_API_KEY'] = userdata.get("NEWS")


**What this does**

Reads secret keys stored in Colab’s userdata and places them into the environment variables GOOGLE_API_KEY and NEWS_API_KEY.

userdata.get("RAGAGENTKEY") returns whatever value you stored under that key in Colab.

**Why use env vars**

Keeps secrets out of code cells.

Other functions can access keys with os.environ.get("NEWS_API_KEY").

If userdata.get() returns None, the environment variable becomes None. Always check for that before using the key.

# ***Creating a Simple News Tool (single-step)***

In [50]:
import os
import requests

def get_latest_news(topic: str) -> str:
    news_key = os.environ.get("NEWS_API_KEY")   # <- FIX
    if not news_key:
        return "NEWS_API_KEY is missing"

    url = (
        f"https://newsapi.org/v2/everything?"
        f"q={topic}&language=en&sortBy=publishedAt&apiKey={news_key}"
    )

    resp = requests.get(url).json()

    if not resp.get("articles"):
        return f"No news found for {topic}"

    titles = []
    for a in resp["articles"][:10]:
        if a.get("title"):
            titles.append(a["title"])

    return "\n".join(f"- {t}" for t in titles)



**What this does step-by-step**

1. news_key = os.environ.get("NEWS_API_KEY") — reads the key you stored earlier.

2. If missing, returns a readable error string.

3. Builds the NewsAPI URL with the topic and the API key.

4. Calls the API with requests.get() and turns the response into JSON.

5. If no articles, returns a helpful message.

6. Gathers up to 5 article titles and returns them joined with new lines.

**Why return a string**

Tools generally return simple primitives (like strings), which are easy to print or feed back to the LLM.


# ***Wrap this function as a LangChain Tool***

In [21]:
from langchain_core.tools import tool

@tool
def news_fetcher(topic: str):
    "Fetch latest news about a topic."
    return get_latest_news(topic)




**What the @tool decorator does**

1. Wraps the Python function in a LangChain Tool/StructuredTool object.

2. LangChain now understands the function name, signature and docstring; the LLM can call it.

3. The wrapper adds runtime methods like .run() and metadata LangChain expects.

# ***Creating Gemini 2.5 Flash model with LangChain***


In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)



**What this does**

1. Creates a LangChain LLM wrapper that calls Gemini (Gemini 2.5 Flash).

2. temperature=0 instructs the model to be deterministic — it prefers factual, less creative outputs.

**Why this object**

1. This object has methods like .invoke(), .bind_tools(), and .generate() depending on LangChain version.

2. It’s the central LLM the agent will use.

# ***Bind the News Tool***

In [54]:
agent = model.bind_tools([news_fetcher])


**What bind_tools does**

1. Returns a new LLM/agent object that knows about the news_fetcher tool.

2. When you call agent.invoke(...), the LLM can decide to call news_fetcher (it will return a structure describing the tool call), but it won’t automatically execute the tool for you.

Why this is “single-pass”

1. The LLM decides in one pass whether to call the tool and returns that decision (no internal loop).

2. You then run the tool separately (manual step) and can feed the output back to the LLM if you need further processing.

# ***Invoke the Agent***

In [48]:
response = agent.invoke("What is the recent news Apple foldable phone")

response


AIMessage(content='', additional_kwargs={'function_call': {'name': 'news_fetcher', 'arguments': '{"topic": "Apple foldable phone"}'}, '__gemini_function_call_thought_signatures__': {'7f673aa0-074e-47f3-bd5b-15edebd7104e': 'CuQBAdHtim8HCbbdvsKPKOkC/0CzBuluyh0hYr+Ztd9z6cBdXfWtgCbYcJwpnMs1ImLN9cFKiwQw4I1wmCsBiqUDF6C4uva9JE/a5OHBRoYohS/oKw4NQGLa6w0+7n7qspkDamysXJ30f+jB4usYd3e/OwABTniXsuaG0ylJYt0mvKrm2kPlq+tNM87pyfx6/9TB9gONXAxhx8pEUlX8OcN5PD2G2PGfNcHel/FEDYQgjA60uxSuANHlCXH3LypEE6R7sivwVvM1PPBlN3iP12xovkfYcsYyIpe9iDTMiLPnr4GnDSFY'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--a2d9d09f-632a-4e16-92de-c81bd850f171-0', tool_calls=[{'name': 'news_fetcher', 'args': {'topic': 'Apple foldable phone'}, 'id': '7f673aa0-074e-47f3-bd5b-15edebd7104e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 49, 'output_tokens': 64, 'total_t

**What to expect in response**

1. response is usually a structured object. Typical important fields:

2. response.output — may contain the LLM’s text if it answered without calling tools.

3. response.tool_calls — a list of tool calls the LLM decided to make. Each call includes tool name and args.

4. response.metadata or other run details depending on the LangChain version.

# ***Then execute the tool's result:***

In [53]:
final_answer = news_fetcher.run(response.tool_calls[0]["args"])
print(final_answer)


- You just crowned a new foldable champion – and it's not Samsung
- Apple to Split iPhone Launches
- Key foldable iPhone hardware reportedly finalized by Apple
- The iPhone Air can’t catch a break: The key designer behind the struggling ultrathin phone just left Apple for an AI startup
- Thieves are returning Android phones because they ‘don’t want no Samsung’
- Peak Android vs. Peak iPhone: Galaxy Fold 7 vs. iPhone 17 Pro Max Showdown!
- iPhone Air designer leaves Apple as Air 2 reportedly targets 2027
- The iPhone Fold Leaks Are Wild: See the 10 Craziest Rumors About Apple’s First Foldable
- The Designer Who Presented the iPhone Air to the World Has Left Apple, Report Says
- iPhone Air: A Foldable iPhone Warm-Up


**What this does**

1. Takes the first tool call the model requested and runs the actual Python tool with the provided args.

2. news_fetcher.run(...) executes get_latest_news(...) internally and returns the string of headlines.

3. print(final_answer) shows the news titles.

**Why it’s manual**

bind_tools + invoke() separates decision (LLM) and execution (your Python), so you control API calls, caching, retries, etc.